<a href="https://colab.research.google.com/github/Guzman2304/IoT_Unab2025/blob/main/IoTParcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install iotc
%pip install setuptools
%pip install asyncio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.5/168.5 kB 6.9 MB/s eta 0:00:00
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62116 sha256=48e83ec585b55ea0baa6b1be22321deb9f902d9a77e13944ae7ff8e1774ba261
  Stored in directory: /root/.cache/pip/wheels/23/d5/af/1f3cbcc350dec9d8e95040f388e0163d132eff0c9a453db659
Successfully built paho-mqtt


In [2]:
import os
import asyncio
import configparser
import sys
import json
import random

from random import randint

from iotc.models import Property, Command

# config = configparser.ConfigParser()
# config.read(os.path.join(os.path.dirname(__file__), "samples.ini"))

# if config["DEFAULT"].getboolean("Local"):
#     sys.path.insert(0, "src")

from iotc import (
    IOTCConnectType,
    IOTCLogLevel,
    IOTCEvents,
    Command,
    CredentialsCache,
    Storage,
)
from iotc.aio import IoTCClient

# Update the following details with your Azure IoT Central credentials
device_id = "2m2jw53h24c"
scope_id = "0ne00FAEEF0"
key = "62rsgS5rQlyyGrZ0o7prgnuCxbPh19+wsdMo1huwdRs="

class MemStorage(Storage):
    def retrieve(self):
        return None

    def persist(self, credentials):
        # a further option would be updating config file with latest hub name
        return None

# optional model Id for auto-provisioning
model_id = None

async def on_props(prop: Property):
    print(f"Received {prop.name}:{prop.value}")
    return True

async def on_commands(command: Command):
    print("Received command {} with value {}".format(command.name, command.value))
    await command.reply()

async def on_enqueued_commands(command: Command):
    print("Received offline command {} with value {}".format(command.name, command.value))

client = IoTCClient(
    device_id,
    scope_id,
    IOTCConnectType.IOTC_CONNECT_DEVICE_KEY,
    key,
    storage=MemStorage(),
)
if model_id != None:
    client.set_model_id(model_id)

client.set_log_level(IOTCLogLevel.IOTC_LOGGING_ALL)
client.on(IOTCEvents.IOTC_PROPERTIES, on_props)
client.on(IOTCEvents.IOTC_COMMAND, on_commands)
client.on(IOTCEvents.IOTC_ENQUEUED_COMMAND, on_enqueued_commands)

async def main():
    await client.connect()
    # await client.send_property({"writeableProp": 50}) # Removed as it might cause issues without a config file

    while not client.terminated():
        if client.is_connected():
            await client.send_telemetry(
                {
                    "Rack1": random.randint(18, 27),#TEMPERATIRA
                    "Rack12": random.randint(40,60),#HUNEDAD
                    "Rack2": random.randint(18, 27),#TEMPERATIRA
                    "Rack21": random.randint(40, 60),#HUNEDAD
                    "Rack3": random.randint(18, 27),#TEMPERATIRA
                    "Rack31": random.randint(40, 60),#HUNEDAD
                    "Seguridad": random.randint(30, 50),#sonido en decibeles
                    "Energia": random.randint(110, 120),#energia en voltios
                    "Energia2": random.randint(0, 30),#Energia en amperios
                    "AireCO2Sala": random.randint(400, 700),#Aire CO2 en sala, medida en ppm
                    "Clima": random.randint(120, 140)#presion para flujo del aire Pa
                },

            )
        await asyncio.sleep(25)

    await client.disconnect()  # Disconnect upon termination


# The original code had asyncio.run(main()) here, which causes an error in Colab.
# We will instead directly await the main coroutine.
await main()

Found cached credentials
Device key: 62rsgS5rQlyyGrZ0o7prgnuCxbPh19+wsdMo1huwdRs=
Device connected to 'iotc-7c709758-1c1a-415d-93cc-591cc1a9aab7.azure-devices.net'
Current twin: {'desired': {'$version': 1}, 'reported': {'$version': 6, 'writeableProp': 50}}
Syncing property '$version'
Sending telemetry message: {'Rack1': 20, 'Rack12': 59, 'Rack2': 24, 'Rack21': 55, 'Rack3': 25, 'Rack31': 51, 'Seguridad': 41, 'Energia': 120, 'Energia2': 11, 'AireCO2Sala': 694, 'Clima': 138}
Sending telemetry message: {'Rack1': 19, 'Rack12': 45, 'Rack2': 21, 'Rack21': 48, 'Rack3': 25, 'Rack31': 44, 'Seguridad': 39, 'Energia': 110, 'Energia2': 21, 'AireCO2Sala': 690, 'Clima': 127}
Sending telemetry message: {'Rack1': 22, 'Rack12': 48, 'Rack2': 25, 'Rack21': 53, 'Rack3': 26, 'Rack31': 55, 'Seguridad': 50, 'Energia': 118, 'Energia2': 18, 'AireCO2Sala': 462, 'Clima': 122}
Sending telemetry message: {'Rack1': 26, 'Rack12': 48, 'Rack2': 23, 'Rack21': 43, 'Rack3': 18, 'Rack31': 53, 'Seguridad': 34, 'Energia': 11

PipelineNotRunning: Cannot execute method - Pipeline is not running